In [18]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras 
from sklearn.model_selection import train_test_split
import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import InputLayer, BatchNormalization,Activation, MaxPool2D
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet101
import tqdm
import keras
import glob
import cv2
import warnings
from keras.preprocessing import image
import numpy as np
warnings.filterwarnings("ignore")
import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)

Num GPUs Available:  0


In [29]:
batch_size=60
img_size=(224,224)
image_generator = ImageDataGenerator(rescale=1/255., validation_split=0) #shear_range =.25, zoom_range =.2, horizontal_flip = True, rotation_range=20)     
train_data = image_generator.flow_from_directory(batch_size=batch_size,directory='F:\Alzheimers-ADNI\\train',shuffle=True,target_size=img_size, subset="training",class_mode='categorical')

image_generator = ImageDataGenerator(rescale=1/255,validation_split=0.7) 
validation_data= image_generator.flow_from_directory(batch_size=batch_size,directory='F:\Alzheimers-ADNI\\test',shuffle=True,target_size=img_size,class_mode='categorical')

submission = image_generator.flow_from_directory(directory='F:\Alzheimers-ADNI\\test',shuffle=False,subset="validation",target_size=img_size, class_mode=None)

Found 1101 images belonging to 5 classes.
Found 195 images belonging to 5 classes.
Found 134 images belonging to 5 classes.


In [30]:
train_data.class_indices

{'Final AD JPEG': 0,
 'Final CN JPEG': 1,
 'Final EMCI JPEG': 2,
 'Final LMCI JPEG': 3,
 'Final MCI JPEG': 4}

In [31]:
batch_size=60
epoch=10
callback= keras.callbacks.EarlyStopping(monitor='val_loss',patience=8,restore_best_weights=True)

In [32]:
rn101 = ResNet101(input_shape=(224,224,3), weights='imagenet', include_top=False)
for layer in rn101.layers:
    layer.trainable = False
x = Flatten()(rn101.output)
x = Dense(128, activation='relu')(x)
out = Dense(5, activation='softmax')(x)

modelrn101= Model(inputs=rn101.input, outputs=out)

# Compiling
modelrn101.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

#Summary
modelrn101.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [33]:
hist_rn101=modelrn101.fit(train_data, epochs=epoch, validation_data=validation_data, callbacks=callback)

Epoch 1/10
19/19 [==============================] - 146s 7s/step - loss: 4.4974 - accuracy: 0.3815 - val_loss: 1.5309 - val_accuracy: 0.1795
Epoch 2/10
19/19 [==============================] - 134s 7s/step - loss: 1.4455 - accuracy: 0.4142 - val_loss: 1.4143 - val_accuracy: 0.4718
Epoch 3/10
19/19 [==============================] - 264s 14s/step - loss: 1.3759 - accuracy: 0.4696 - val_loss: 1.4854 - val_accuracy: 0.4667
Epoch 4/10
19/19 [==============================] - 292s 16s/step - loss: 1.4029 - accuracy: 0.4650 - val_loss: 1.3927 - val_accuracy: 0.4667
Epoch 5/10
19/19 [==============================] - 174s 9s/step - loss: 1.3623 - accuracy: 0.4696 - val_loss: 1.4118 - val_accuracy: 0.4667
Epoch 6/10
19/19 [==============================] - 154s 8s/step - loss: 1.3968 - accuracy: 0.4723 - val_loss: 1.3832 - val_accuracy: 0.4667
Epoch 7/10
19/19 [==============================] - 226s 12s/step - loss: 1.3681 - accuracy: 0.4614 - val_loss: 1.4137 - val_accuracy: 0.4667
Epoch 8/10

In [ ]:
loss_rn101,accuracy_rn101= modelrn101.evaluate(validation_data)

4/4 [==============================] - 25s 5s/step - loss: 1.4589 - accuracy: 0.4667


In [35]:
import pickle
# save the iris classification model as a pickle file
model_pkl_file = "rn101ADNI.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(modelrn101, file)